In [2]:
import argparse
import os
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import scipy.io
import scipy.misc
import numpy as np
import pandas as pd
import PIL
import tensorflow as tf
from keras import backend as K
from keras.layers import Input, Lambda, Conv2D
from keras.models import load_model, Model

from yad2k.models.keras_yolo import yolo_head, yolo_boxes_to_corners, preprocess_true_boxes, yolo_loss, yolo_body
from yolo_utils import *

Using TensorFlow backend.


In [3]:
def yolo_filter_boxes(box_confidence, boxes, box_class_probs, threshold=.6):
	"""
	Filters YOLO boxes by thresholding on object and class confidence
	:param box_confidence: tensor of shape(19, 19, 5, 1)
	:param boxes: tensor of shape(19, 19, 5, 4)
	:param box_class_prob: tensor of shape(19,19,5,80)
	:param threshold: real value
	:return: 
	
	"""
	#step1 compute box scores
	box_scores = box_confidence * box_class_probs
	
	#step2: Find the box_classes thanks to to max box_scores, keep track of the corresponding score
	box_classes = K.argmax(box_scores, axis=-1)
	box_class_scores = K.max(box_scores, axis=-1, keepdims=False)
	
	#step3: Create a filter mask based on "box_class_scores" by using "threshold". The mask should
	# have the same dimension as box_class_scores, and be True for the boxex you want to keep
	filtering_mask = box_class_scores >= threshold
	
	# Step4: Apply the mask to socres, boxes and classes
	scores = tf.boolean_mask(box_class_scores, filtering_mask)
	boxes = tf.boolean_mask(boxes, filtering_mask)
	classes = tf.boolean_mask(box_classes, filtering_mask)
	
	return scores, boxes, classes
	

In [4]:
with tf.Session() as sess:
	box_confidence = tf.random_normal([19,19,5,1], mean=1, stddev=4, seed=1)
	boxes = tf.random_normal([19,19,5,4], mean=1, stddev=4, seed=1)
	box_class_probs = tf.random_normal([19,19,5,80], mean=1, stddev=4, seed=1)
	scores, boxes, classes = yolo_filter_boxes(box_confidence, boxes, box_class_probs,threshold=0.5)
	print(str(scores[2].eval()))
	print(str(boxes[2].eval()))
	print(str(classes[2].eval()))
	print(str(scores.shape))
	print(str(boxes.shape))
	print(str(classes.shape))
	
	sess.close()

10.750582
[ 8.426533   3.2713668 -0.5313436 -4.9413733]
7
(?,)
(?, 4)
(?,)


In [5]:
def iou(box1, box2):
	"""
	comput iou
	:param box1: tuple, first box,(x1, y1, x2, y2)
	:param box2: tuple, second box, (x1, y1, x2, y2)
	:return: iou float
	"""
	#step1: calculate the (y1, x1,y2,x2),inter_area
	xi1 = max(box1[0], box2[0])
	yi1 = max(box1[1], box2[1])
	xi2 = min(box1[2], box2[2])
	yi2 = min(box1[3], box2[3])
	inter_area = (yi2-yi1) * (xi2-xi1)
	
	#step2: Calculate the Union area by using Formula: Union(A,B)=A+B-Inter(A,B)
	box1_area = (box1[3]-box1[1]) * (box1[2]-box1[0])
	box2_area = (box2[3]-box2[1]) * (box2[2]-box2[0])
	union_area = box1_area + box2_area - inter_area
	
	#step3: Calculate the IoU
	iou = inter_area / union_area
	
	return iou

In [6]:
box1 = (2,1,4,3)
box2 = (1,2,3,4)
print('iou='+str(iou(box1, box2)))



iou=0.14285714285714285


In [7]:
def yolo_non_max_suppression(scores, boxes, classes, max_boxes=10, iou_threshold=0.5):
	"""
	Applies Non-max suppression to set of boxes
	:param scores: tensor of shape(None,) output of yolo_fileter_boxes()
	:param boxes: tensor of shape(None,) output of yolo_fileter_boxes() that have been scaled
	to the image size
	:param classes: tensor of shape(None,),output of yolo_filter_boxes()
	:param max_boxes: integer, maximum number of predict boxes you`d like
	:param iou_threshold: real_value, "intersection over union" threshold for NMS filtering
	:return: 
	scores:tensor of shape(,None), predicted score for each box
	boxes:tensor of shape(,None), predicted box coordinates
	classes:tensor of shape(,None), predicted class for each box
	"""
	max_boxes_tensor = K.variable(max_boxes, dtype='int32')
	K.get_session().run(tf.variables_initializer([max_boxes_tensor]))
	
	#Use tf.image.non_max_suppression() to get the list of indices corresponding to boxes you keep
	nms_indices = tf.image.non_max_suppression(boxes, scores, max_boxes, iou_threshold)
	
	#Use K.gather() to select only nms_indices from scores, boxes and calsses
	scores = K.gather(scores, nms_indices)
	boxes = K.gather(boxes, nms_indices)
	classes = K.gather(classes, nms_indices)
	
	return scores, boxes, classes
	

In [8]:
with tf.Session() as test_b:
	scores = tf.random_normal([54,], mean=1, stddev=4, seed=1)
	boxes = tf.random_normal([54,4], mean=1, stddev=4, seed=1)
	classes = tf.random_normal([54,], mean=1, stddev=4, seed=1)
	scores, boxes, classes = yolo_non_max_suppression(scores, boxes, classes)
	print(str(scores[2].eval()))
	print(str(boxes[2].eval()))
	print(str(classes[2].eval()))
	print(str(scores.eval().shape))
	print(str(boxes.eval().shape))
	print(str(classes.eval().shape))

Instructions for updating:
Colocations handled automatically by placer.
6.938395
[-5.299932    3.1379814   4.450367    0.95942086]
-2.2452729
(10,)
(10, 4)
(10,)


In [9]:
def yolo_eval(yolo_outputs, image_shape=(720,180),max_boxes=10, score_threshold=0.6,
              iou_threshold=0.5):
	"""
	Converts the outputs of YOLO encoding (a lot of boxes) to your predict boxes along with
	their scores, box coordinates and classes
	:param yolo_outputs: output of the encoding model(for image_shape of(608, 608, 3), contains 4 tensors
	:param image_shape:tensor of shape(2,) containing the input shape, in this notebook we use (608,608) 
	:param max_boxes: integer, maximum number of predicted boxes you`d like
	:param score_threshold: real_value
	:param iou_threshole: real_value
	:return: 
	scores -- tensor of shape(None,), predict score for each box
	boxes -- tensor of shape(None,4), predict box coordinates
	classes -- tensor of shape(None,), predict class for each box
	"""
	#Retrieve outputs of the YOLO model
	box_confidence, box_xy, box_wh, box_class_probs = yolo_outputs
	
	#Convert boxes to be ready for filtering functiong
	boxes = yolo_boxes_to_corners(box_xy, box_wh)
	
	#Use ont of the functions you`ve implemented to perform Score-filtering with 
	# a threshold of score_threshold)
	scores, boxes, classes  = yolo_filter_boxes(box_confidence, boxes, box_class_probs, score_threshold)
	
	#Scale boxes back to original image shape.
	boxes = scale_boxes(boxes, image_shape)
	
	#Use one of the functions you`ve implemented to perform Non-max suppression with a threshold
	# of iou_threshold)
	score, boxes, classes = yolo_non_max_suppression(scores, boxes, classes, max_boxes, iou_threshold)
	
	return scores, boxes, classes
	

In [11]:
with tf.Session() as test_c:
	yolo_outputs = (tf.random_normal([19,19,5,1], mean=1, stddev=4, seed=1),
	                tf.random_normal([19,19,5,2], mean=1, stddev=4, seed=1),
	                tf.random_normal([19,19,5,2], mean=1, stddev=4, seed=1),
	                tf.random_normal([19,19,5,80], mean=1, stddev=4, seed=1),)
	scores, boxes, classes = yolo_eval(yolo_outputs)
	print(str(scores[2].eval()))
	print(str(boxes[2].eval()))
	print(str(classes[2].eval()))
	print(str(scores.eval().shape))
	print(str(boxes.eval().shape))
	print(str(classes.eval().shape))
	

TypeError: Input 'y' of 'Mul' Op has type int32 that does not match type float32 of argument 'x'.